In [1]:
import pandas as pd
import numpy as np

In [3]:
col=['name','score']
PN=pd.read_table('/root/rlap-main_fe_gnn/test.csv',sep='\t',)
PN.rename(columns={'Unnamed: 0':'name','score':'score'},inplace=True)
PN

,name,score
0,BRAF,13.031697
1,TP53,10.024441
2,PIK3CA,6.800762
3,PTEN,6.615622
4,APC,5.953482
...,...,...
7690,GUCY1B1,-6.114473
7691,OR8J1,-6.244836
7692,CSN2,-6.316767
7693,MAIP1,-6.472750


In [7]:
##读取各个网络的得分文件 ['name','score']
col=['name','score']
# GN=pd.read_table('/root/gp/GGNet.txt',sep='\t',header=None,names=col)
GN=pd.read_table('/root/rlap-main_fe_gnn/predicted_scores/predicted_socres_GGNet.txt',sep='\t',)
GN.rename(columns={'Unnamed: 0':'name','score':'score'},inplace=True)
PN=pd.read_table('../predicted_scores/predicted_socres_PathNet.txt',sep='\t')
# PN=pd.read_table('/root/HGDC-master/trained_models/predicted_gene_list_PathNet.txt',sep='\t',names=col)
PN.rename(columns={'Unnamed: 0':'name','score':'score'},inplace=True)
PPN=pd.read_table('../predicted_scores/predicted_socres_PPNet.txt',sep='\t')
PPN.rename(columns={'Unnamed: 0':'name','score':'score'},inplace=True)
# GN.to_csv('GN.csv')
# PN.to_csv('PN.csv')
# PPN.to_csv('PPN.csv')
PPN

,name,score
0,TP53,7.029612
1,BRAF,6.774221
2,PTEN,4.756589
3,PIK3CA,4.637312
4,APC,3.262257
...,...,...
11390,GALNTL6,-5.992653
11391,GALNT9,-6.132866
11392,SPRR2G,-6.419472
11393,LCE3C,-6.504500


#### 处理Potential drivers、canonical_drivers、nondrivers

In [4]:
###读取Potential drivers和canonical_drivers
cancer_drivers=pd.read_csv('/root/rlap-main_fe_gnn/process_score/NCG_cancerdrivers_annotation_supporting_evidence.tsv',sep='\t')[['symbol','type']]
col=['name']
c_drivers=pd.read_table('/root/autodl-tmp/HGDC-master/data/796_drivers.txt',names=col)
c_drivers.drop_duplicates(inplace=True)
####读取canonical_drivers
d=cancer_drivers[cancer_drivers['type']=='Canonical Cancer Driver']['symbol']
d.drop_duplicates(inplace=True)
d=pd.DataFrame(d)
###读取Potential drivers
P_d=cancer_drivers[cancer_drivers['type']!='Canonical Cancer Driver']['symbol']
P_d=pd.DataFrame(P_d)
P_d.drop_duplicates(inplace=True)
##读取nodrivers
column_name=['name']
N_d=pd.read_table('/root/autodl-tmp/HGDC-master/data/2187_nondrivers.txt',sep='\t')
N_d=pd.DataFrame(N_d)
N_d.drop_duplicates(inplace=True)
# c_drivers,N_d,P_d

/tmp/ipykernel_62853/1067180294.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.drop_duplicates(inplace=True)


#### 对得分进行sigmoid预处理

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
# GN['nscore']=GN['score'].apply(sigmoid)
# PPN['score']=PPN['score']
# PPN['nscore']=PPN['score'].apply(sigmoid)
PN['nscore']=PN['score']
PN['nscore']=PN['nscore'].apply(sigmoid)
# PN_other['nscore']=PN_other['score'].apply(sigmoid)
# PN_other
PN_T=PN[PN['nscore']>=0.5]
PN

,name,score,nscore
0,BRAF,13.031697,0.999998
1,TP53,10.024441,0.999956
2,PIK3CA,6.800762,0.998888
3,PTEN,6.615622,0.998663
4,APC,5.953482,0.997410
...,...,...,...
7690,GUCY1B1,-6.114473,0.002206
7691,OR8J1,-6.244836,0.001937
7692,CSN2,-6.316767,0.001803
7693,MAIP1,-6.472750,0.001543


In [16]:
####计算PathNet网络
PN_T.head()
###compute canonical_drivers scores
intersection = set(PN_T.iloc[:, 0]).intersection(set(c_drivers.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PN=PN_T[PN_T['name'].isin(intersection_series)]
print(len(cd_PN))
#compute potential_driver scores
intersection = set(PN_T.iloc[:, 0]).intersection(set(P_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PN=PN_T[PN_T['name'].isin(intersection_series)]
print(len(cd_PN))
####compute non_drivers scores
intersection = set(PN_T.iloc[:, 0]).intersection(set(N_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PN=PN_T[PN_T['name'].isin(intersection_series)]
print(len(cd_PN))
####compute rest_gene scores 
intersection = set(PN_T.iloc[:, 0]).difference(set(N_d.iloc[:, 0]),set(c_drivers.iloc[:, 0]),set(P_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PN=PN_T[PN_T['name'].isin(intersection_series)]
print(len(cd_PN))

42
43
3
14


### 计算网络中的drivers、nondrivers和rest的平均得分

In [105]:
####计算GGNet网络
GN.head()
###compute canonical_drivers scores
intersection = set(GN.iloc[:, 0]).intersection(set(d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_GN=GN[GN['name'].isin(intersection_series)]
print('canonical_drivers scores',cd_GN['nscore'].mean())
print(cd_GN)
#compute potential_driver scores
intersection = set(GN.iloc[:, 0]).intersection(set(P_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
P_GN=GN[GN['name'].isin(intersection_series)]
print('potential_driver scores',P_GN['nscore'].mean())
print(P_GN)
####compute non_drivers scores
intersection = set(GN.iloc[:, 0]).intersection(set(N_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_GN=GN[GN['name'].isin(intersection_series)]
print('non_drivers scores',cd_GN['nscore'].mean())
####compute rest_gene scores 
intersection = set(GN.iloc[:, 0]).difference((set(N_d.iloc[:, 0])|set(c_drivers.iloc[:, 0])|set(P_d.iloc[:, 0])))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_GN=GN[GN['name'].isin(intersection_series)]
print('rest_score',cd_GN['nscore'].mean())

canonical_drivers scores 0.18156223262069757
           name      score    nscore
0          BRAF  12.658229  0.999997
1          TP53   9.349344  0.999913
2        PIK3CA   7.113059  0.999186
3          PTEN   6.290550  0.998150
4          CDH1   5.812744  0.997020
...         ...        ...       ...
10527      AFDN  -5.556605  0.003847
10880   CYSLTR2  -6.027498  0.002406
11035     PREX2  -6.331305  0.001777
11057     RSPO2  -6.416093  0.001632
11164  APOBEC3B  -7.143483  0.000789

[519 rows x 3 columns]
potential_driver scores 0.10584953773872147
         name      score    nscore
0        BRAF  12.658229  0.999997
1        TP53   9.349344  0.999913
2      PIK3CA   7.113059  0.999186
3        PTEN   6.290550  0.998150
4        CDH1   5.812744  0.997020
...       ...        ...       ...
11111  ZNF697  -6.663847  0.001275
11137   LUZP2  -6.820190  0.001090
11144  SEMA5B  -6.894985  0.001012
11145  SEMA5A  -6.895181  0.001012
11180  ZNF180  -7.771149  0.000422

[2233 rows x 3 columns

In [136]:
-39.44243010851752/(-26.106611392585737-39.44243010851752-14.92755549647059-45.90891718693627)
# -14.92755549647059/(-26.106611392585737-39.44243010851752-14.92755549647059-45.90891718693627)

0.31208030732806563

In [8]:
PN

,name,score,nscore
0,BRAF,13.031697,0.999998
1,TP53,10.024441,0.999956
2,PIK3CA,6.800762,0.998888
3,PTEN,6.615622,0.998663
4,APC,5.953482,0.997410
...,...,...,...
7690,GUCY1B1,-6.114473,0.002206
7691,OR8J1,-6.244836,0.001937
7692,CSN2,-6.316767,0.001803
7693,MAIP1,-6.472750,0.001543


In [9]:
####计算PathNet网络
PN.head()
###compute canonical_drivers scores
intersection = set(PN.iloc[:, 0]).intersection(set(c_drivers.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PN=PN[PN['name'].isin(intersection_series)]
print('canonical_drivers scores',cd_PN['nscore'].mean())
#compute potential_driver scores
intersection = set(PN.iloc[:, 0]).intersection(set(P_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PN=PN[PN['name'].isin(intersection_series)]
print('potential_driver scores',cd_PN['nscore'].mean())
####compute non_drivers scores
intersection = set(PN.iloc[:, 0]).intersection(set(N_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PN=PN[PN['name'].isin(intersection_series)]
print('non_drivers scores',cd_PN['nscore'].mean())
####compute rest_gene scores 
intersection = set(PN.iloc[:, 0]).difference(set(N_d.iloc[:, 0]),set(c_drivers.iloc[:, 0]),set(P_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PN=PN[PN['name'].isin(intersection_series)]
print('rest_score',cd_PN['nscore'].mean())

canonical_drivers scores 0.18139982184666378
potential_driver scores 0.11958538318100649
non_drivers scores 0.045743720957312425
rest_score 0.05602236605051925


In [10]:
####计算PPNet网络
PPN.head()
###compute canonical_drivers scores
intersection = set(PPN.iloc[:, 0]).intersection(set(c_drivers.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PPN=PPN[PPN['name'].isin(intersection_series)]
print('canonical_drivers scores',cd_PPN['nscore'].mean())
#compute potential_driver scores
intersection = set(PPN.iloc[:, 0]).intersection(set(P_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PPN=PPN[PPN['name'].isin(intersection_series)]
print('potential_driver scores',cd_PPN['nscore'].mean())
####compute non_drivers scores
intersection = set(PPN.iloc[:, 0]).intersection(set(N_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PPN=PPN[PPN['name'].isin(intersection_series)]
print('non_drivers scores',cd_PPN['nscore'].mean())
####compute rest_gene scores 
intersection = set(PN.iloc[:, 0]).difference(set(N_d.iloc[:, 0]),set(c_drivers.iloc[:, 0]),set(P_d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素
cd_PPN=PPN[PPN['name'].isin(intersection_series)]
print('rest_score',cd_PPN['nscore'].mean())

canonical_drivers scores 0.1766958718376823
potential_driver scores 0.10019966464994734
non_drivers scores 0.03615288232447701
rest_score 0.04623702543631152


In [66]:
ss=pd.read_csv('/root/rlap-main_fe_gnn/predicted_scores/data/density/density.csv')
col=['name']
c_drivers=pd.read_table('../../HGDC-master/data/796_drivers.txt',names=col)
# cd=pd.read_table('/root/rlap-main_fe_gnn/process_score/NCG_cancerdrivers_annotation_supporting_evidence.tsv')
# cd.head()
c_drivers
ss

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,name,GN_c,PN_c,PPN_c,PPN_sc,GN_sc,PN_sc,AVG_C,AVG_sc
0,0,NaN,NaN,NaN,CDH1,189.0,157,122.0,2.953108,5.812744,2.845120,156.000000,3.870324
1,1,1.0,1.0,1.0,MTOR,329.0,120,114.0,0.075088,0.625895,0.081530,187.666667,0.260838
2,2,2.0,2.0,2.0,KMT2B,9.0,10,1.0,0.341093,0.392477,-0.557127,6.666667,0.058814
3,3,3.0,3.0,3.0,HRAS,33.0,407,408.0,0.212715,1.168430,2.058906,282.666667,1.146684
4,4,4.0,4.0,4.0,TP53,238.0,558,324.0,9.585825,9.349344,8.015185,373.333333,8.983451
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,79,79.0,79.0,79.0,SRC,100.0,431,370.0,-0.535594,-2.561039,-0.449163,300.333333,-1.181932
80,80,80.0,80.0,80.0,NOTCH2,755.0,73,57.0,0.118944,-0.533432,0.309529,295.000000,-0.034987
81,81,81.0,81.0,81.0,PRKDC,70.0,16,2.0,0.089635,-0.819371,-0.275019,29.333333,-0.334918
82,82,82.0,82.0,82.0,ESR1,19.0,248,150.0,0.243743,-1.346252,0.218019,139.000000,-0.294830


In [71]:
intersection = set(ss.iloc[:, 4]).difference(set(d.iloc[:, 0]))
# 将交集转换为 Series 并显示
intersection_series = pd.Series(list(intersection))
intersection_series  # 显示交集的前几个元素


0      HSPA8
1      PRKCA
2      NCOA3
3      PRKDC
4      SYNE1
5     TNRC6A
6        HGF
7      IGF1R
8     ITGA2B
9        FYN
10      TLR4
11     AKAP9
12     MUC16
13     KMT2B
14     KAT2B
15    TNRC6B
16    IQGAP1
17      SOS1
18       VWF
19      MED1
20     HLA-A
21     PLCG2
22       FN1
dtype: object

In [5]:
import os
import glob
import pandas as pd
import numpy as np

# 设置包含所有网络预测分数文件的基本路径
base_path = '/root/rlap-main_fe_gnn/cancer_Net'

# 函数：加载网络文件并返回排序的数据帧
def load_and_rank(file_path):
    # 加载文件
    cols=['gene','score']
    df = pd.read_table(file_path,sep="\t",names=cols)
    # 根据分数对基因进行排序，分数越高排名越靠前
    df.sort_values(by='score', ascending=False, inplace=True)
    # 创建新列 'rank' 以存储排名
    df['rank'] = range(1, len(df) + 1)
    return df
def calculate_rank_product_and_mean_score(ggnet_rank, pathnet_rank, ppnet_rank):
    
    # 计算 RankProduct，即排名的几何平均数
    rank_product = (ggnet_rank * pathnet_rank * ppnet_rank) ** (1/3)
    
    # 计算平均分数
    # mean_score = (ggnet_score + pathnet_score + ppnet_score)
    
    return rank_product
# 示例：计算特定癌症类型的 RankProduct
def get_cancer_types(base_path):
    # 假设每个网络的文件夹中的癌症类型文件名格式是统一的
    example_network_path = os.path.join(base_path, 'GGNet/Result/')
    # 列出所有文件并提取癌症类型
    return [f.split('_')[2] for f in os.listdir(example_network_path) if f.endswith('.txt')]
ggnet_file ='/root/rlap-main_fe_gnn/predicted_scores/predicted_socres_GGNet.txt'
pathnet_file ='/root/rlap-main_fe_gnn/predicted_scores/predicted_socres_PathNet.txt' 
ppnet_file ='/root/rlap-main_fe_gnn/predicted_scores/predicted_socres_PPNet.txt' 

# 加载并排序
ggnet_df = load_and_rank(ggnet_file)
pathnet_df = load_and_rank(pathnet_file)
ppnet_df = load_and_rank(ppnet_file)
# 将 DataFrame 中的 'score' 列转换为数值类型
ggnet_df['score'] = pd.to_numeric(ggnet_df['score'], errors='coerce')
pathnet_df['score'] = pd.to_numeric(pathnet_df['score'], errors='coerce')
ppnet_df['score'] = pd.to_numeric(ppnet_df['score'], errors='coerce')
# 合并三个网络的排名
merged_df = pd.merge(ggnet_df, pathnet_df, on='gene', suffixes=('_ggnet', '_pathnet'))
merged_df = pd.merge(merged_df, ppnet_df, on='gene')
merged_df['rank_product'] = calculate_rank_product_and_mean_score(merged_df['rank_ggnet'], merged_df['rank_pathnet'], merged_df['rank'])
# 根据 RankProduct 排序
merged_df.sort_values(by='rank_product', inplace=True)
# 显示结果
merged_df[['gene', 'rank_product']].dropna().head(100).to_csv('./all_gene_rank_product.csv')
